In [ ]:
#VIMCO
#FINAL CODE
import pandas as pd
import re

# --- Step 1: Read Excel ---
#df = pd.read_excel(r"C:\Users\Admin\Downloads\PO4675.xlsx")
#df = pd.read_excel(r"C:\Users\Admin\Downloads\PO_BHAKTI_4656.xlsx")   # BHAKTI
df = pd.read_excel(r"C:\Users\Admin\Downloads\VIMCO_PO.xlsx")  # VIMCO

# --- Step 2: Select relevant columns ---
selected_columns = ['#', 'Item #', 'Vendor Item #', 'Description', 'Size', 'Qty.','Dia Qlty']
df = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df.rename(columns={
    '#': 'SrNo',
    'Item #': 'SKU',
    'Vendor Item #': 'StyleCode',
    'Description': 'CustomerProductionInstruction',
    'Size': 'ItemSize',
    'Qty.': 'OrderQty',
}, inplace=True)

# --- Step 4: Ask user for input columns ---
item_po_no = input("Enter ItemPoNo number: ")
order_group = input("Enter OrderGroup value: ")
user_priority = input("Enter Priority (press Enter to use default '5 day'): ")
priority_value = user_priority if user_priority.strip() != "" else "5 day"

# --- Step 5: Functions for calculated columns ---
def map_metal(text):
    if pd.isna(text):
        return ''
    text = str(text).upper()
    if '14KY' in text:
        return 'G14Y'
    elif '14KW' in text:
        return 'G14W'
    elif '18KY' in text:
        return 'G18Y'
    elif '18KW' in text:
        return 'G18W'
    elif '10KY' in text:
        return 'G10Y'
    elif '10KW' in text:
        return 'G10W'
    elif 'PT' in text:
        return 'PC95'
    else:
        return ''

def map_tone(metal):
    if metal == 'PC95':
        return 'PT'
    elif metal:
        return metal[-1]
    else:
        return ''

def metal_text(metal):
    mapping = {
        "G14Y": "14K YELLOW GOLD",
        "G14W": "14K WHITE GOLD",
        "G18Y": "18K YELLOW GOLD",
        "G18W": "18K WHITE GOLD",
        "G10Y": "10K YELLOW GOLD",
        "G10W": "10K WHITE GOLD",
        "PC95": "PLATINUM"
    }
    return mapping.get(metal, "")

def extract_dia_quality(text):
    if pd.isna(text):
        return ""
    text = str(text).upper()
    if "LGD" in text:
        after_lgd = text.split("LGD", 1)[1].strip()
        words = after_lgd.split()
        return ' '.join(words[:2])
    return ""

def special_remarks(row):
    metal_descr = metal_text(row['Metal'])
    dia_quality = extract_dia_quality(row['CustomerProductionInstruction'])
    return f"{row['OrderGroup']},{row['SKU']},{metal_descr}, DIA QLTY - {dia_quality}"

def design_production_instruction(row):
    tone = str(row['Tone']).upper()
    cpi = str(row['CustomerProductionInstruction']).upper() if pd.notna(row['CustomerProductionInstruction']) else ''
    semi_present = "SEMI" in cpi

    if tone == "W" and semi_present:
        return "SEMI MOUNT, WHITE RODIUM"
    elif tone == "Y" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "W" and not semi_present:
        return "WHITE RODIUM"
    elif tone == "Y" and not semi_present:
        return "NO RODIUM"
    elif tone == "PT" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "PT" and not semi_present:
        return "NO RODIUM"
    else:
        return ""

# --- New helper functions for StampInstruction ---
def extract_stone_weight(text):
    if pd.isna(text):
        return ""
    text = str(text)
    match = re.search(r'(\d+\.\d+|\d+)\s*(CT|CARAT|CTS|CTW|A|PCS)?', text, re.IGNORECASE)
    if match:
        return match.group(1)
    return ""

def metal_stamp_text(metal):
    mapping = {
        "G14Y": "14K",
        "G14W": "14K",
        "G18Y": "18K",
        "G18W": "18K",
        "G10Y": "10K",
        "G10W": "10K",
        "PC95": "PT950"
    }
    return mapping.get(metal, "")

def generate_stamp_instruction(row):
    metal_text_val = metal_stamp_text(row['Metal'])
    stone_weight = extract_stone_weight(row['CustomerProductionInstruction'])
    if metal_text_val and stone_weight:
        return f"{metal_text_val} V ON ONE SIDE AND {stone_weight} A ON OTHER SIDE"
    elif metal_text_val:
        return f"{metal_text_val} V ON ONE SIDE"
    else:
        return ""

# --- Step 6: Create all new columns ---
df['OrderItemPcs'] = ''
df['Metal'] = df['CustomerProductionInstruction'].apply(map_metal)
df['Tone'] = df['Metal'].apply(map_tone)
df['ItemPoNo'] = item_po_no
df['ItemRefNo'] = ''
df['StockType'] = ''
df['Priority'] = priority_value
df['MakeType'] = ''
df['StampInstruction'] = df.apply(generate_stamp_instruction, axis=1)
df['OrderGroup'] = order_group
df['Certificate'] = '' 
df['SpecialRemarks'] = df.apply(special_remarks, axis=1)
df['DesignProductionInstruction'] = df.apply(design_production_instruction, axis=1)

# --- Step 6c: Add new columns after SKU ---
new_cols_after_sku = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]

for col in new_cols_after_sku:
    df[col] = ''  # Initialize empty or default value

# --- ZR style default ItemSize = 7 ---
df.loc[df['StyleCode'].astype(str).str.upper().str.startswith('ZR'), 'ItemSize'] = 7

# --- Extract size from SKU if it ends with SZ9, SZ6.5, etc. ---
def extract_size_from_sku(sku):
    if pd.isna(sku):
        return None
    sku = str(sku).upper()
    match = re.search(r'SZ(\d+(\.\d+)?)$', sku)
    if match:
        return match.group(1)
    return None

df['ExtractedSize'] = df['SKU'].apply(extract_size_from_sku)
df['ItemSize'] = df['ItemSize'].fillna(df['ExtractedSize'])
df.drop(columns=['ExtractedSize'], inplace=True)

# --- Step 7: Reorder columns ---
final_columns = [
    'SrNo', 'StyleCode', 'ItemSize', 'OrderQty', 'OrderItemPcs', 'Metal', 'Tone',
    'ItemPoNo', 'ItemRefNo', 'StockType', 'Priority', 'MakeType',
    'CustomerProductionInstruction', 'SpecialRemarks', 'DesignProductionInstruction',
    'StampInstruction', 'OrderGroup', 'Certificate', 'SKU'
] + new_cols_after_sku

df = df[final_columns]

# --- Step 8: Export to CSV ---
#df.to_csv(r'C:\Users\Admin\Desktop\VIMCO_Cleaned.csv', index=False)

# --- Step 9: Preview final DataFrame ---
df.head()


In [4]:
#VIMCO
#FINAL CODE
import pandas as pd
import re

# --- Step 1: Read Excel ---
df = pd.read_excel(r"C:\Users\Admin\Downloads\VIMCO_PO.xlsx")  # VIMCO

# --- Step 2: Select relevant columns ---
selected_columns = ['#', 'Item #', 'Vendor Item #', 'Description', 'Size', 'Qty.', 'Dia Qlty']
df = df[selected_columns].copy()

# --- Step 3: Rename columns ---
df.rename(columns={
    '#': 'SrNo',
    'Item #': 'SKU',
    'Vendor Item #': 'StyleCode',
    'Description': 'CustomerProductionInstruction',
    'Size': 'ItemSize',
    'Qty.': 'OrderQty',
    'Dia Qlty': 'DiaQuality'
}, inplace=True)

# --- Clean up StyleCode to remove everything after '-' ---
df['StyleCode'] = df['StyleCode'].astype(str).str.split('-').str[0].str.strip()

# --- Step 4: Ask user for input columns ---
item_po_no = input("Enter ItemPoNo number: ")
order_group = input("Enter OrderGroup value: ")
user_priority = input("Enter Priority (press Enter to use default '5 day'): ")
priority_value = user_priority if user_priority.strip() != "" else "5 day"

# --- Step 5: Functions for calculated columns ---
def map_metal(text):
    if pd.isna(text):
        return ''
    text = str(text).upper()
    if '14KY' in text:
        return 'G14Y'
    elif '14KW' in text:
        return 'G14W'
    elif '18KY' in text:
        return 'G18Y'
    elif '18KW' in text:
        return 'G18W'
    elif '10KY' in text:
        return 'G10Y'
    elif '10KW' in text:
        return 'G10W'
    elif 'PT' in text:
        return 'PC95'
    else:
        return ''

def map_tone(metal):
    if metal == 'PC95':
        return 'PT'
    elif metal:
        return metal[-1]
    else:
        return ''

def metal_text(metal):
    mapping = {
        "G14Y": "14K YELLOW GOLD",
        "G14W": "14K WHITE GOLD",
        "G18Y": "18K YELLOW GOLD",
        "G18W": "18K WHITE GOLD",
        "G10Y": "10K YELLOW GOLD",
        "G10W": "10K WHITE GOLD",
        "PC95": "PLATINUM"
    }
    return mapping.get(metal, "")

def special_remarks(row):
    metal_descr = metal_text(row['Metal'])
    dia_quality = str(row['DiaQuality']).strip().upper() if pd.notna(row['DiaQuality']) else ""
    return f"{row['OrderGroup']},{row['SKU']},{metal_descr}, DIA QLTY - {dia_quality}"

def design_production_instruction(row):
    tone = str(row['Tone']).upper()
    cpi = str(row['CustomerProductionInstruction']).upper() if pd.notna(row['CustomerProductionInstruction']) else ''
    semi_present = "SEMI" in cpi

    if tone == "W" and semi_present:
        return "SEMI MOUNT, WHITE RODIUM"
    elif tone == "Y" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "W" and not semi_present:
        return "WHITE RODIUM"
    elif tone == "Y" and not semi_present:
        return "NO RODIUM"
    elif tone == "PT" and semi_present:
        return "SEMI MOUNT, NO RODIUM"
    elif tone == "PT" and not semi_present:
        return "NO RODIUM"
    else:
        return ""

# --- New helper functions for StampInstruction ---
def extract_stone_weight(text):
    if pd.isna(text):
        return ""
    text = str(text)
    match = re.search(r'(\d+\.\d+|\d+)\s*(CT|CARAT|CTS|CTW|A|PCS)?', text, re.IGNORECASE)
    if match:
        return match.group(1)
    return ""

def metal_stamp_text(metal):
    mapping = {
        "G14Y": "14K",
        "G14W": "14K",
        "G18Y": "18K",
        "G18W": "18K",
        "G10Y": "10K",
        "G10W": "10K",
        "PC95": "PT950"
    }
    return mapping.get(metal, "")

def generate_stamp_instruction(row):
    metal_text_val = metal_stamp_text(row['Metal'])
    stone_weight = extract_stone_weight(row['CustomerProductionInstruction'])
    if metal_text_val and stone_weight:
        return f"{metal_text_val} V ON ONE SIDE AND {stone_weight} A ON OTHER SIDE"
    elif metal_text_val:
        return f"{metal_text_val} V ON ONE SIDE"
    else:
        return ""

# --- Step 6: Create all new columns ---
df['OrderItemPcs'] = ''
df['Metal'] = df['CustomerProductionInstruction'].apply(map_metal)
df['Tone'] = df['Metal'].apply(map_tone)
df['ItemPoNo'] = item_po_no
df['ItemRefNo'] = ''
df['StockType'] = ''
df['Priority'] = priority_value
df['MakeType'] = ''
df['StampInstruction'] = df.apply(generate_stamp_instruction, axis=1)
df['OrderGroup'] = order_group
df['Certificate'] = '' 
df['SpecialRemarks'] = df.apply(special_remarks, axis=1)
df['DesignProductionInstruction'] = df.apply(design_production_instruction, axis=1)

# --- Step 6c: Add new columns after SKU ---
new_cols_after_sku = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]
for col in new_cols_after_sku:
    df[col] = ''

# --- ZR style default ItemSize = 7 ---
df.loc[df['StyleCode'].astype(str).str.upper().str.startswith('ZR'), 'ItemSize'] = 7

# --- Extract size from SKU if it ends with SZ9, SZ6.5, etc. ---
def extract_size_from_sku(sku):
    if pd.isna(sku):
        return None
    sku = str(sku).upper()
    match = re.search(r'SZ(\d+(\.\d+)?)$', sku)
    if match:
        return match.group(1)
    return None

df['ExtractedSize'] = df['SKU'].apply(extract_size_from_sku)
df['ItemSize'] = df['ItemSize'].fillna(df['ExtractedSize'])
df.drop(columns=['ExtractedSize'], inplace=True)

# --- Step 7: Reorder columns ---
final_columns = [
    'SrNo', 'StyleCode', 'ItemSize', 'OrderQty', 'OrderItemPcs', 'Metal', 'Tone',
    'ItemPoNo', 'ItemRefNo', 'StockType', 'Priority', 'MakeType',
    'CustomerProductionInstruction', 'SpecialRemarks', 'DesignProductionInstruction',
    'StampInstruction', 'OrderGroup', 'Certificate', 'SKU', 'DiaQuality'
] + new_cols_after_sku
df = df[final_columns]

# --- Step 8: Export to CSV ---
df_selected.drop(columns=['DiaQuality'], inplace=True) 
df.to_csv(r'C:\Users\Admin\Desktop\VIMCO_Cleaned.csv', index=False)

# --- Step 9: Preview final DataFrame ---
df.head()


Enter ItemPoNo number:  5499
Enter OrderGroup value:  
Enter Priority (press Enter to use default '5 day'):  REG


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,SKU,DiaQuality,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,RG0001164X,6.5,3,,G14Y,Y,5499,,,...,R6200-060YD,A,,,,,,,,
